In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-08-22 13:17:08--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  36.5MB/s    in 7.6s    

2020-08-22 13:17:17 (32.9 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip -q concrete_data_week4.zip 

In [3]:
import os
os.listdir('concrete_data_week4')

['test', '.DS_Store', 'valid', 'train']

<a id="item42"></a>

In [4]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16


In [5]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [6]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

valid_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')




Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [7]:
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)
num_epochs = 2
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 310s 1s/step - loss: 0.1240 - accuracy: 0.9556 - val_loss: 0.0302 - val_accuracy: 0.9936
Epoch 2/2
301/301 [==============================] - 309s 1s/step - loss: 0.0229 - accuracy: 0.9952 - val_loss: 0.0180 - val_accuracy: 0.9964


In [13]:
model_vgg16 = model

<a id="item43"></a>

In [15]:


from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

model = Sequential()
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)
num_epochs = 2
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)


model_resnet50 = model


Epoch 1/2
301/301 [==============================] - 272s 905ms/step - loss: 0.0354 - accuracy: 0.9868 - val_loss: 0.0077 - val_accuracy: 0.9980
Epoch 2/2
301/301 [==============================] - 271s 901ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.0055 - val_accuracy: 0.9985


In [16]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.


In [17]:
model_vgg16.evaluate_generator(test_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.01479776669293642, 0.9980000257492065]

In [18]:
model_resnet50.evaluate_generator(test_generator)

[0.002374266739934683, 1.0]

<a id="item44"></a>

In [23]:
from numpy import argmax
index_to_label = {v: k for k, v in test_generator.class_indices.items()}

In [24]:
predictions_vgg16 = model_vgg16.predict_generator(test_generator)

for pred_probs in predictions_vgg16[:5]:
    print(index_to_label.get((argmax(pred_probs))))

negative
negative
negative
negative
negative


In [25]:
predictions_resnet50 = model_resnet50.predict_generator(test_generator)

for pred_probs in predictions_resnet50[:5]:
    print(index_to_label.get((argmax(pred_probs))))



negative
negative
negative
negative
negative
